In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import pathlib

python_path = pathlib.Path('.').absolute().parent/'python'
os.sys.path.insert(1, str(python_path))

In [3]:
## This is demo for kuka reaching a with mpc and diff_qp
## Author : Avadesh Meduri
## Date : 25/02/2022

import time
import numpy as np
import pinocchio as pin
from robot_properties_kuka.config import IiwaConfig
from matplotlib import pyplot as plt

import meshcat
import meshcat.transformations as tf
import meshcat.geometry as g

from vocam.diff_pin_costs import DiffFrameTranslationCost, DiffFrameVelocityCost
from vocam.inverse_qp import IOC

import torch
from torch.autograd import Function
from torch.nn import functional as F
from pinocchio.robot_wrapper import RobotWrapper

<frozen importlib._bootstrap>:219: RuntimeWarning: to-Python converter for boost::shared_ptr<hpp::fcl::CollisionGeometry> already registered; second conversion method ignored.


In [4]:
urdf_path = "/home/ameduri/devel/workspace/robot_properties/example-robot-data/robots/panda_description/urdf/panda.urdf"
meshes_path = "/home/ameduri/devel/workspace/robot_properties/example-robot-data/robots/panda_description/meshes/"
robot = RobotWrapper.BuildFromURDF(
            urdf_path, meshes_path)
robot = IiwaConfig.buildRobotWrapper()


In [5]:
model, data = robot.model, robot.data
# f_id = model.getFrameId("panda_hand_joint")
f_id = model.getFrameId("EE")

# j_id = [f_id]
# o_id = model.getFrameId("A7")

In [6]:
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [7]:
class DiffFramePlacementCost(Function):
    """
    This cost provides gradients wrt joint position and velocity for the final end effector/frame placement
    """
    @staticmethod
    def forward(ctx, state, model, data, f_id, M_des):
        """
        Input:
            state : vector (q, dq)
            model : pinocchio robot model
            data : pinocchio robot data
            f_id : frame id for which derivatives are desired
        """
        state = state.double().detach().numpy()
        nq, nv = model.nq, model.nv
        pin.forwardKinematics(model, data, state[0:nq], state[nq:nq + nv], np.zeros(nv))
        pin.updateFramePlacements(model, data)
        
        se3_error = pin.SE3(M_des).inverse() * data.oMf[f_id]
        error = np.array(pin.log6(se3_error))
        
        J_error = pin.Jlog6(se3_error)       
        J = pin.computeFrameJacobian(model, data, state[0:nq], f_id, pin.ReferenceFrame.LOCAL)
        
        ctx.J = torch.tensor(J_error @ J)
        
        return torch.tensor(error)

    @staticmethod
    def backward(ctx, grad):
        
        jac = ctx.J
        fk_q = jac.t()@grad # derivative wrt joint positions
        fk_dq = torch.zeros(len(fk_q)) # derivative wrt joint velocities
        
        return torch.hstack((fk_q, fk_dq)), None, None, None, None
        

In [8]:
class DiffFrameOrientationCost(Function):
    """
    This cost provides gradients wrt joint position and velocity for the final end effector/frame placement
    """
    @staticmethod
    def forward(ctx, state, model, data, f_id, M_des):
        """
        Input:
            state : vector (q, dq)
            model : pinocchio robot model
            data : pinocchio robot data
            f_id : frame id for which derivatives are desired
        """
        state = state.double().detach().numpy()
        nq, nv = model.nq, model.nv
        pin.forwardKinematics(model, data, state[0:nq], state[nq:nq + nv], np.zeros(nv))
        pin.updateFramePlacements(model, data)
        
        se3_error = np.linalg.inv(M_des) * data.oMf[f_id].rotation
        error = np.array(pin.log3(se3_error))
        
        J_error = pin.Jlog3(se3_error)       
        J = pin.computeFrameJacobian(model, data, state[0:nq], f_id, pin.ReferenceFrame.LOCAL)
        
        ctx.J = torch.tensor(J_error @ J[3:6])
        
        return torch.tensor(error)

    @staticmethod
    def backward(ctx, grad):
        
        jac = ctx.J
        fk_q = jac.t()@grad # derivative wrt joint positions
        fk_dq = torch.zeros(len(fk_q)) # derivative wrt joint velocities
        
        return torch.hstack((fk_q, fk_dq)), None, None, None, None

In [9]:
dtc = DiffFrameTranslationCost.apply
dvc = DiffFrameVelocityCost.apply
dfc = DiffFramePlacementCost.apply

def quadratic_loss(q_pred, goal, nq, n_col, j):
    
    O_des = np.array(pin.utils.rpyToMatrix(-np.pi/4, np.pi/2, 0))
#     error = doc(q_pred[-2*nq:], model, data, f_id, O_des)
#       wt = 1e1*np.eye(3)
  
    M_des = np.eye(4,4)
    M_des[0:3,0:3] = O_des
    M_des[0:3,3] = goal
    wt = 1e3*np.eye(6)
    wt[3,3] = 5e1
    wt[4,4] = 5e1
    wt[5,5] = 5e1
    error = dfc(q_pred[-2*nq:], model, data, f_id, M_des)
    
    loss = error.t()@torch.tensor(wt)@error
    
#     loss += 5.0e1*torch.linalg.norm(dtc(q_pred[-2*nq:], model, data, f_id) - goal)
    
    loss += 2.5e0*torch.linalg.norm(dvc(q_pred[-2*nq:], torch.zeros(nq), model, data, f_id)) # asking for zero velocity
    loss += 1e-3*torch.linalg.norm(q_pred[-2*nq:-nq]) # joint regularization
    
    for i in range(n_col):    
        loss += 2e0 * torch.linalg.norm(dtc(q_pred[(3*i)*nq: (3*i+2)*nq], model, data, f_id) - goal)
        loss += 5e-1*torch.linalg.norm(dvc(q_pred[(3*i)*nq: (3*i+2)*nq], q_pred[(3*i+2)*nq:(3*i+3)*nq], model, data, f_id)) # asking for zero velocity
        loss += 1e-2*torch.linalg.norm(q_pred[(3*i+2)*nq: (3*i+3)*nq]) # control regularization
        loss += 2e-1*torch.linalg.norm(q_pred[(3*i+1)*nq: (3*i+2)*nq]) # velocity regularization
        loss += 5e-3*torch.linalg.norm(q_pred[(3*i)*nq: (3*i+1)*nq]) # joint regularization
        loss += 2e-3*torch.linalg.norm(q_pred[(3*i)*nq+3: (3*i+1)*nq])
        loss += 4e-3*torch.linalg.norm(q_pred[(3*i)*nq+5])
        
#         if i < n_col - 1:
#             loss += 5e-2*torch.linalg.norm(torch.subtract(q_pred[(3*i+2)*nq: (3*i+3)*nq], \
#                                                           q_pred[(3*i+5)*nq: (3*i+6)*nq]))

    return loss

In [10]:
obs_rad = [0.5, 0.35, 0.4]

In [11]:
nq = model.nq
nv = model.nv

n_col = 5
u_max = [3.5,4.5,2.5, 2.5, 1.5, 1.5, 1.0]

lr = 1e-1
eps = 80

# q_des = np.hstack(((np.pi/4)*(np.random.randint(0, 2, size = 5)), np.zeros(2)))

q_des_arr = np.array([[2.1789238e-02,  3.3214998e-01, -1.4518893e-04, -8.7141126e-01,
                          6.0329604e-01, -1.3965217e-03,  1.4794523e-04],
                      [1.3737, 0.9711, 1.6139, 1.2188, 1.5669, 0.1236, 0.2565]])
goal_arr = torch.tensor([[0.4, -0.3, 0.5], [0.4, 0.3, 0.5]]) #torch.tensor([[0.6, 0.4, 0.5]]) #[0.6, 0.4, 1.0]

# q_init = np.hstack(((np.pi/8)*np.random.choice(list(range(-2, -1)) + list(range(1, 3)), size=(4)), np.zeros(3)))

x_init = np.zeros(2*nq)
x_init[0:nq] = q_des_arr[0] + 0.3*2*(np.random.rand(nq) - 0.5)
# x_init[5] = np.pi
x_init[0] -= 2*0.5*(np.random.rand(1) - 0.5)
x_init[2] -= 2*0.3*(np.random.rand(1) - 0.5)
x_init[nq:] = 0.7*2*(np.random.rand(nv) - 0.5)

x_in = x_init

In [12]:
for k in range(1):
    r = 0.3*np.random.rand(1) + 0.5
    rint = k % 2
    print(rint)
#     theta = rint*(0.1*np.pi*(np.random.rand(1)) + 0.65*np.pi) + (1-rint)*(0.1*np.pi*(np.random.rand(1)) + 0.25*np.pi)
#     goal = torch.squeeze(torch.tensor([r*np.sin(theta), r*np.cos(theta), 0.15*np.random.rand(1)+0.15]))
    goal = torch.tensor([0.6, 0, 0.5])

    viz.viewer["box"].set_object(g.Sphere(0.05), 
                             g.MeshLambertMaterial(
                                 color=0xff22dd,
                                 reflectivity=0.8))
    viz.viewer["box"].set_transform(tf.translation_matrix(goal.detach().numpy()))

    for j in range(25):

        ioc = IOC(n_col, nq, u_max, 0.05, eps = 1.0, isvec=True)
        optimizer = torch.optim.Adam(ioc.parameters(), lr=lr)


        i = 0
        loss = 1000.
        old_loss = 10000.

        while loss > 0.03 and i < eps and abs(old_loss - loss) > 5e-5:
            x_pred = ioc(x_in) 

            old_loss = loss

    #         loss = quadratic_loss(x_pred, goal, nq, n_col)
            loss = quadratic_loss(x_pred, goal, nq, n_col, j)
            print("Index :" + str(i) + " loss is : " + str(loss.detach().numpy()), end = '\r', flush = True)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            i += 1

        x_pred = ioc(x_in).detach().numpy()
        
        for i in range(n_col+1):
            q = x_pred[3*nq*i:3*nq*i + nq]
            dq = x_pred[3*nq*i + nq:3*nq*i + 2*nq]
            if i < n_col:
                ddq = x_pred[3*nq*i + 2*nq:3*nq*i + 3*nq]
            pin.forwardKinematics(model, data, q, dq, np.zeros(nv))
            pin.updateFramePlacements(model, data)
            viz.display(q)
            time.sleep(0.1)

        x_in = torch.tensor(x_pred[-2*nq:])

0


KeyboardInterrupt: 

In [ ]:
data.oMf[f_id]

In [20]:
O_des = np.array(pin.utils.rpyToMatrix(np.pi/4.0,0, 0))  
M_des = np.eye(4,4)
M_des[0:3,0:3] = O_des
M_des[0:3,3] = goal
# np.round(M_des,3)

In [27]:
from scipy.spatial.transform import Rotation as R
ori_mat = R.from_matrix(pin.utils.rpyToMatrix(-np.pi/4, np.pi/2, 0)).as_quat()

In [26]:
ori_mat.as_quat()

array([-0.27059805,  0.65328148,  0.27059805,  0.65328148])